In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Import Library

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import pandas as pd
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
import math
from sklearn.metrics import mean_squared_error

In [ ]:
df = pd.read_csv('/kaggle/input/bitcoin-btc-usd-stock-dataset/BTC-USD_stock_data.csv',index_col='Date',parse_dates=['Date'])
df.head()

In [ ]:
def plot_pred(test,pred):
    plt.plot(test, color='r',label='Aktual Harga Bitcoin')
    plt.plot(pred, color='b',label='Prediksi Harga Bitcoin')
    plt.title('Prediksi Harga Bitcoin')
    plt.xlabel('Waktu')
    plt.ylabel('Harga Bitcoin')
    plt.legend()
    plt.show()
    
def return_rmse(test,pred):
    rmse = math.sqrt(mean_squared_error(test,pred))
    print('Root MSE = {}.'.format(rmse))

In [ ]:
# Cek Missing Values
train_set = df[:'2020'].iloc[:,1:2].values
test_set = df['2021':].iloc[:,1:2].values

In [ ]:
# Pilih atribut 'High' untuk harga

df['High'][:'2020'].plot(figsize=(10,8),legend=True)
df['High']['2021':].plot(figsize=(10,8),legend=True)
plt.legend(['Training set(2020)','Test set(2021)'])
plt.title('Harga BTC')
plt.show()

In [ ]:
# Skala untuk Training set

sc = MinMaxScaler(feature_range=(0,1))
train_set_sc = sc.fit_transform(train_set)

In [ ]:
train_set_sc.shape

In [ ]:
# Karena LSTM menyimpan status memori jangka panjang, kita membuat struktur data dengan 60 langkah waktu dan 1 keluaran
# Jadi untuk setiap elemen set pelatihan, kita memiliki 60 elemen set pelatihan sebelumnya

x_train = []
y_train = []
for i in range(60,1461):
    x_train.append(train_set_sc[i-60:i,0])
    y_train.append(train_set_sc[i,0])
x_train,y_train = np.array(x_train),np.array(y_train)

In [ ]:
# Membentuk ulang shape x_train

x_train = np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))

In [ ]:
# Arsitektur LSTM
model_lstm = Sequential()

#1
model_lstm.add(LSTM(units=50,return_sequences=True,input_shape=(x_train.shape[1],1)))
model_lstm.add(Dropout(0.2))

#2
model_lstm.add(LSTM(units=50,return_sequences=True))
model_lstm.add(Dropout(0.2))

#3
model_lstm.add(LSTM(units=50,return_sequences=True))
model_lstm.add(Dropout(0.2))

#4
model_lstm.add(LSTM(units=50))
model_lstm.add(Dropout(0.2))

#Output
model_lstm.add(Dense(units=1))

#Compile RNN
model_lstm.compile(optimizer='rmsprop',loss='mean_squared_error')

#Fit training set
model_lstm.fit(x_train,y_train,epochs=50,batch_size=32)

Dengan nilai loss nya semakin kecil, maka hasilnya semakin baik

In [ ]:
# Sekarang untuk menyiapkan set pengujian dengan cara yang sama seperti set pelatihan.

# Berikut ini telah dilakukan sehingga 60 bagian set pengujian memiliki 60 nilai sebelumnya yang tidak mungkin diperoleh kecuali kita mengambil seluruh
# Data atribut 'Tinggi' untuk diproses

df_total = pd.concat((df['High'][:'2020'],df['High']['2021':]),axis=0)
inputs = df_total[len(df_total)-len(test_set)-60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
inputs.shape

In [ ]:
# Mempersiapkan x_test dan prediksi harga

x_test=[]
for i in range(60,1435):
    x_test.append(inputs[i-60:i,0])
x_test = np.array(x_test)
x_test = np.reshape(x_test,(x_test.shape[0],x_test.shape[1],1))
pred_harga = model_lstm.predict(x_test)
pred_harga = sc.inverse_transform(pred_harga)
plot_pred(test_set,pred_harga)

In [ ]:
# Evaluasi model

return_rmse(test_set,pred_harga)

### Gated Recurrent Units (GRU)

In [ ]:
#Arsitektur GRU
lstm_gru = Sequential()

#1
lstm_gru.add(GRU(units=50,return_sequences=True,input_shape=(x_train.shape[1],1),activation='tanh'))
lstm_gru.add(Dropout(0.2))

#2
lstm_gru.add(GRU(units=50,return_sequences=True,input_shape=(x_train.shape[1],1),activation='tanh'))
lstm_gru.add(Dropout(0.2))

#3
lstm_gru.add(GRU(units=50,return_sequences=True,input_shape=(x_train.shape[1],1),activation='tanh'))
lstm_gru.add(Dropout(0.2))

#4
lstm_gru.add(GRU(units=50,activation='tanh'))
lstm_gru.add(Dropout(0.2))

#Output
lstm_gru.add(Dense(units=1))

#Compile RNN
lstm_gru.compile(optimizer=SGD(learning_rate=0.01, decay=1e-7, momentum=0.9, nesterov=False),loss='mean_squared_error')

# Fit untuk training test
lstm_gru.fit(x_train,y_train,epochs=50,batch_size=150)

In [ ]:
# x_test dan prediksi harga

x_test = []
for i in range(60,1435):
    x_test.append(inputs[i-60:i,0])
x_test = np.array(x_test)
x_test = np.reshape(x_test,(x_test.shape[0],x_test.shape[1],1))
pred_harga_gru = lstm_gru.predict(x_test)
pred_harga_gru = sc.inverse_transform(pred_harga_gru)
plot_pred(test_set,pred_harga_gru)

In [ ]:
# Evaluasi model GRU

return_rmse(test_set,pred_harga_gru)